## 两种读取txt文件的方法对比

In [17]:
f1 = open("./dict.txt","r", encoding="utf8")
lines1 = f1.read() # 读到str类型
print(type(lines1))
print("lines1 =", lines1)

f2 = open("./dict.txt","r", encoding="utf8")
lines2 = f2.readlines() # 读到list类型
print(type(lines2))
print("lines2 =", lines2)
# 使用readlines读取到list可以进行逐行处理
for line in lines2:
    print(line)

<class 'str'>
lines1 = 江泽民
国家主席
1998年
1997年
1995年
1999年
<class 'list'>
lines2 = ['江泽民\n', '国家主席\n', '1998年\n', '1997年\n', '1995年\n', '1999年']
江泽民

国家主席

1998年

1997年

1995年

1999年


## 为每个句子添加头BOS，与尾EOS

In [30]:
import jieba
import re

#语料句子
sentence_ori="""研究生物很有意思。他大学时代是研究生物的。生物专业是他的首选目标。（他是研究生。） 
他一直很喜欢生物。生物专业真的很不错。"""
#测试句子

sentence_test="研究生物专业是他的首选目标"

#任务：完成对2-gram模型的建立，计算测试句子概率并输出结果

punc = r"""\n ！  ？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.#$%&'()*+,-./:;<=>?@[\]^_`{|}~“”？，！【】（）、。：；’‘……￥·"""

def Modify(s):
    #将结尾标点符号截掉
    if s[-1] in (r"[%s]+"%punc):
        s = s[:-1]  #截取字符串从开头到倒数一个字符的子串
    if s[0] in (r"[%s]+"%punc):
        s = s[1:]
    #添加起始符BOS和终止符EOS   
    s_modify1 = re.sub(r"[%s]+"%punc, "EOS BOS", s)   ## r'\w+'为正则表达式，匹配多个英文单词或者数字  
    s_modify2="BOS"+s_modify1+"EOS"
    return s_modify2

train_Modifys=Modify(sentence_ori)
print(train_Modifys)

test_Modifys =Modify(sentence_test)
# print(test_Modifys)

BOS研究生物很有意思EOS BOS他大学时代是研究生物的EOS BOS生物专业是他的首选目标EOS BOS他是研究生EOS BOS他一直很喜欢生物EOS BOS生物专业真的很不错EOS


## 博主所用分词方法

In [26]:
def Partition_Statistics(s, dicts = None):
    jieba.suggest_freq(('BOS','EOS'), True)#分开
    s = jieba.cut(s, HMM = False)  #精确模式，自动计算的词频在使用 HMM 新词发现功能时可能无效,所以设为False
    format_s = ",".join(s)
    #将词按","分割后依次填入数组
    lists = [i.strip() for i in format_s.split(",") if i not in punc]
    # s = jieba.lcut(s, HMM = False)
    #统计词频
    if dicts != None:
        for word in lists:
            if word not in dicts:
                dicts[word] = 1
            else:
                dicts[word] += 1               
    return lists , dicts


train_seg,train_count = Partition_Statistics(train_Modifys,dicts = {})
print(train_seg)
print(train_count)
test_seg,test_count = Partition_Statistics(test_Modifys,dicts = {})
print(test_seg)
print(test_count)

['BOS', '研究', '生物', '很', '有意思', 'EOS', 'BOS', '他', '大学', '时代', '是', '研究', '生物', '的', 'EOS', 'BOS', '生物', '专业', '是', '他', '的', '首选', '目标', 'EOS', 'BOS', '他', '是', '研究生', 'EOS', 'BOS', '他', '一直', '很', '喜欢', '生物', 'EOS', 'BOS', '生物', '专业', '真的', '很', '不错', 'EOS']
{'BOS': 6, '研究': 2, '生物': 5, '很': 3, '有意思': 1, 'EOS': 6, '他': 4, '大学': 1, '时代': 1, '是': 3, '的': 2, '专业': 2, '首选': 1, '目标': 1, '研究生': 1, '一直': 1, '喜欢': 1, '真的': 1, '不错': 1}
['BOS', '研究', '生物', '专业', '是', '他', '的', '首选', '目标', 'EOS']
{'BOS': 1, '研究': 1, '生物': 1, '专业': 1, '是': 1, '他': 1, '的': 1, '首选': 1, '目标': 1, 'EOS': 1}


## 更简单的分词同时加入自定义词典

In [27]:
def WordCount(s):
    jieba.load_userdict("./dict.txt")#加载自定义字典
    jieba.suggest_freq(('BOS','EOS'), True) # 分开
    word_list = jieba.lcut(s, cut_all=False)  # 分词
    while ' ' in word_list:
        word_list.remove(' ')
    #统计词频
    dicts = {}
    for word in word_list:
        if word not in dicts:
            dicts[word] = 1
        else:
            dicts[word] += 1
    return word_list , dicts

In [28]:
def CompareList(ori_list,test_list):
    #申请空间
    count_list=[0]*(len(test_list)-1)
    #遍历测试的字符串
    for i in range(0, len(test_list)-1):
        #遍历语料字符串，因为是二元语法，不用比较语料字符串的最后一个字符
        for j in range(0,len(ori_list)):
            #如果测试的第一个词和语料的第一个词相等则比较第二个词
            if test_list[i]==ori_list[j]:
                if test_list[i+1]==ori_list[j+1]:
                    print(j)
                    count_list[i]+=1
    return count_list

def Probability(test_list,count_list,ori_dict):
    #概率值为p
    p=1
    for i in range(len(count_list)): 
        p *=(float(count_list[i])/float(ori_dict[test_list[i]]))
    return p


count_list = CompareList(train_seg, test_seg)
print(count_list)
p = Probability(train_seg,count_list,train_count)
print(p)


0
1
11
16
37
17
18
19
20
21
22
[1, 2, 2, 1, 1, 1, 1, 1, 1]
0.00015432098765432096


In [29]:
import re
import numpy as np

def removePunctuation(sentence_str):
    '''给定字符串，进行特殊符号过滤，以及小写转换
    Args:
        sentence_str (str): 句子
    
    Returns:
        sentence_str.lower() (str): 预处理过的句子
    '''
    punctuation = '.!,;:?\-"\''
    sentence_str = re.sub(r'[{}]+'.format(punctuation),'',sentence_str)
    return sentence_str.lower()

def get_sentence(filename):
    '''从给定文件中获取句子
    Args:
        filename (str): 语料库的文件名
    
    Returns:
        sentences_list (list): 1-D，存放了所有经过预处理后的句子
    '''
    sentences_list = []
    with open(filename, encoding='utf-8') as f:
        for line in f.readlines():
            sentence_str = removePunctuation(line.strip().split('|')[-1])
            sentences_list.append('<s> ' + sentence_str + ' </s>')
    return sentences_list

def count_word(sentences_list):
    '''给定大量句子，统计出所有单词出现的频次
    Args:
        sentences_list (list): 所有经过预处理后的句子
    
    Returns:
        wordcount_dict (dict): 键是str类型，表示单词；值是int类型，表示次数，例如{'the': 1234}
    '''
    wordcount_dict = {} 
    for sentence_str in sentences_list:
        for word in sentence_str.split():
            if word in wordcount_dict:
                wordcount_dict[word] += 1
            else:
                wordcount_dict[word] = 1
    return wordcount_dict

def word2idx(wordcount_dict):
    '''构建单词到索引的映射与逆映射
    Args:
        wordcount_dict (dict): 键是str类型，表示单词；值是int类型，表示次数
    
    Returns:
        word2idx_dict (dict): 键是str类型，表示单词；值是int类型，表示索引，例如{'the': 0}
        idx2word_dict (dict): 键是int类型，表示索引；值是str类型，表示单词，例如{0: 'the'}
    '''
    word2idx_dict = {}
    idx2word_dict = {}
    for idx, word in enumerate(list(wordcount_dict.keys())):
        word2idx_dict[word] = idx
        idx2word_dict[idx] = word
    return word2idx_dict, idx2word_dict

def c_table(word2idx_dict, sentences_list, smooth=False):
    '''构建两个单词连续出现的频次矩阵
    Args:
        word2idx_dict (dict): 键是str类型，表示单词；值是int类型，表示索引
        sentences_list (list): 所有经过预处理后的句子
        smooth (bool): 是否进行加一平滑
    
    Returns:
        c_table_np (numpy): 2-D，c_table_np[i][j] = a表示 前一个索引为i的词和当前索引为j的词 同时出现的次数为a
    '''
    n = len(word2idx_dict) # 单词个数
    c_table_np = np.zeros((n, n)) # n*n 全0矩阵
    for sentence_str in sentences_list:
        words_list = sentence_str.split() # ['i', 'like', 'apple']
        for i in range(1, len(words_list)):
            w_i = word2idx_dict[words_list[i]] # w_i
            w_j = word2idx_dict[words_list[i-1]] # w_{i-1}
            c_table_np[w_j][w_i] += 1
    
    if smooth: # 加一平滑
        c_table_np[c_table_np == 0] = 1
    
    return c_table_np

def compute_bigram_table(c_table_np, wordcount_dict):
    '''构建bigram概率矩阵
    Args:
        c_table_np (numpy): bigram频次矩阵
        wordcount_dict (dict): 所有单词出现的次数
    
    Returns:
        c_table_np / count_np[:, None] (numpy): 2-D，bigram概率矩阵
    '''
    count_np = np.array(list(wordcount_dict.values())) # [800, 900, 788, ...]
    return c_table_np / count_np[:, None]

def compute_sentence_bigram(bigram_table_np, word2idx_dict, sentences_list):
    '''计算每个句子的bigram概率
    Args:
        bigram_table_np (numpy): bigram概率矩阵
        word2idx_dict (dict): 单词到索引的映射
        sentences_list (list): 预处理后的句子
    
    Returns:
        scores_list (list): 所有句子的bigram概率
    '''
    scores_list = []
    for sentence_str in sentences_list:
        words_list = sentence_str.split()
        score = 1
        for i in range(1, len(words_list)):
            w_i = word2idx_dict[words_list[i]] # w_i
            w_j = word2idx_dict[words_list[i-1]] # w_{i-1}
            score *= bigram_table_np[w_j][w_i]
        scores_list.append(score)
    return scores_list

if __name__ == '__main__':
    sentences_list = get_sentence('./finish.txt')
    wordcount_dict = count_word(sentences_list)
    word2idx_dict, idx2word_dict = word2idx(wordcount_dict)
    c_table_np = c_table(word2idx_dict, sentences_list, True)
    bigram_table_np = compute_bigram_table(c_table_np, wordcount_dict)
    scores_list = compute_sentence_bigram(bigram_table_np, word2idx_dict, sentences_list)
    print(scores_list[:10])

[4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05, 4.3357613596947624e-05]


In [2]:
import jieba
import re
#语料句子
sentence_ori="研究生物很有意思。他大学时代是研究生物的。生物专业是他的首选目标。他是研究生。"
#测试句子

sentence_test="明确而言，国家法律对这种擅自拆除承重墙装修的当事人应受到何种行政处罚和民事责任均有规定。"

#任务：完成对2-gram模型的建立，计算测试句子概率并输出结果


punc = r"""！  ？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.#$%&'()*+,-./:;<=>?@[\]^_`{|}~“”？，！【】（）、。：；’‘……￥·"""


punctuation_map = dict((ord(char), "EOSBOS") for char in punc)  
new_s=sentence_ori.translate(punctuation_map)
print(new_s)


def Modify(s):
    #将结尾标点符号截掉
    if s[-1] in (r"[%s]+"%punc):
        s = s[:-1]  #截取字符串从开头到倒数一个字符的子串

    #添加起始符BOS和终止符EOS   
    s_modify1 = re.sub(r"[%s]+"%punc, "EOS BOS", s)   ## r'\w+'为正则表达式，匹配多个英文单词或者数字  
    s_modify2="BOS"+s_modify1+"EOS"
    return s_modify2

train_Modifys=Modify(sentence_ori)
print(train_Modifys)

test_Modifys =Modify(sentence_test) 

def Partition_Statistics(s, dicts = None):
    jieba.suggest_freq(('BOS','EOS'), True)#分开
    s = jieba.cut(s, HMM = False)  #精确模式，自动计算的词频在使用 HMM 新词发现功能时可能无效,所以设为False
    format_s = ",".join(s)
    #将词按","分割后依次填入数组
    lists = [i.strip() for i in format_s.split(",") if i  not in punc ]
    #统计词频
    if dicts != None:
        for word in lists:
            if word not in dicts:
                dicts[word] = 1
            else:
                dicts[word] += 1               
    return lists , dicts


train_seg,train_count = Partition_Statistics(train_Modifys,dicts = {})
print(train_seg  )
print(train_count)
test_seg,test_count = Partition_Statistics(test_Modifys,dicts = {})

def CompareList(ori_list,test_list):
    #申请空间
    count_list=[0]*(len(test_list)-1)
    #遍历测试的字符串
    for i in range(0, len(test_list)-1):
        #遍历语料字符串，因为是二元语法，不用比较语料字符串的最后一个字符
        for j in range(0,len(ori_list)-1):
            #如果测试的第一个词和语料的第一个词相等则比较第二个词
            if test_list[i]==ori_list[j]:
                if test_list[i+1]==ori_list[j+1]:
                    count_list[i]+=1
    return count_list

def Probability(test_list,count_list,ori_dict):
    #概率值为p
    p=1
    for i in range(len(count_list)): 
        p *=(float(count_list[i])/float(ori_dict[test_list[i]]))
    return p


count_list = CompareList(train_seg, test_seg)
print(count_list)
p = Probability(train_seg,count_list,train_count)
print(p)



研究生物很有意思EOSBOS他大学时代是研究生物的EOSBOS生物专业是他的首选目标EOSBOS他是研究生EOSBOS
BOS研究生物很有意思EOS BOS他大学时代是研究生物的EOS BOS生物专业是他的首选目标EOS BOS他是研究生EOS
['BOS', '研究', '生物', '很', '有意思', 'EOS', 'BOS', '他', '大学', '时代', '是', '研究', '生物', '的', 'EOS', 'BOS', '生物', '专业', '是', '他', '的', '首选', '目标', 'EOS', 'BOS', '他', '是', '研究生', 'EOS']
{'BOS': 4, '研究': 2, '生物': 3, '很': 1, '有意思': 1, 'EOS': 4, '他': 3, '大学': 1, '时代': 1, '是': 3, '的': 2, '专业': 1, '首选': 1, '目标': 1, '研究生': 1}
[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.0
